In [4]:
import networkx as nx
import json
import nltk
import pandas as pd
import re
import matplotlib.pyplot as plt 
import gensim
import community
import pyLDAvis.gensim

import warnings
warnings.filterwarnings('ignore')
pyLDAvis.enable_notebook()

from IPython.display import display
from gensim import corpora, models
from string import punctuation
from nltk.corpus import stopwords
from datetime import datetime
from textblob import TextBlob  
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer as Vectorizer
from nltk  import TweetTokenizer
from gensim.models import CoherenceModel 
from gensim.test.utils import common_corpus, common_dictionary
from gensim.models import HdpModel
from networkx.algorithms.community import greedy_modularity_communities

import numpy as np
import pyLDAvis
import matplotlib.pyplot as plt
nltk.download('stopwords')
pd.set_option('display.max_colwidth', -1)


[nltk_data] Downloading package stopwords to /Users/and/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#Metod para el sentimental Análisis y dibujar la grafica
def sentimentalAnalis(twitText):
    analisis = TextBlob(str(twitText))
    laridad = analisis.sentiment
    laridad = laridad.polarity
    return laridad


In [6]:
def join_all_files_in_one_df(files):
    array_data_frame = []
    for file in files:
        array_data_frame.append(pd.read_json(file, orient='columns'))
    df = pd.concat(array_data_frame)
    return df

In [7]:
def clean_data_from_json(df, ntwits):
    # Load the first sheet of the JSON file into a data frame
    popularidad_list = []
    usernames = []
    sentimentOfTwritUser = []
    
    if ntwits != 0:
        df = df[:ntwits]
        
    user_id = df['user'].tolist()
    twit = df['text'].tolist()
    for usr, twt in zip(user_id, twit):
        usernames.append(eval(str(usr))['screen_name'])
        sntAnalisisReturn = sentimentalAnalis(twt)
        popularidad_list.append(sntAnalisisReturn)        
        sentimentOfTwritUser.append(sntAnalisisReturn)
    numeros_list = range(1,len(twit)+1)
    df=df.assign(screen_name=usernames)
    df=df.assign(sentiment=sentimentOfTwritUser)
    
    data = df['text'].tolist()
    cleanData = []
    for t in data:
        cleanData.append(tweet_clean(t))
    numeros_list = np.asarray(numeros_list)
    return [cleanData, df, numeros_list, sentimentOfTwritUser, len(twit)]

In [8]:
def GraficarDatos(numeros_list, popularidad_list, numero, theTitle):
    axes = plt.gca()
    axes.set_ylim([-1, 2]) 
    
    plt.scatter(numeros_list, popularidad_list)

    popularidadPromedio = (sum(popularidad_list))/(len(popularidad_list))
    popularidadPromedio = "{0:.0f}%".format(popularidadPromedio * 100)
    time  = datetime.now().strftime("")
    # time  = datetime.now().strftime("A : %H:%M\n El: %m-%d-%y")
    plt.text(0, 1.25, 
             "Sentimiento promedio:  " + str(popularidadPromedio) + "\n" + time, 
             fontsize=12, 
             bbox = dict(facecolor='none', 
                         edgecolor='black', 
                         boxstyle='square, pad = 1'))
    
    plt.title(theTitle)
    plt.xlabel("Numero de tweets")
    plt.ylabel("Sentimiento")
    plt.show()

In [9]:
def relaciones_network(df):
    graph=nx.DiGraph()
    userName = df['screen_name'].tolist()
    snetiment = df['screen_name'].tolist()
    
    #Rellenamos el grafo con el nombre d
    for usr, snt in zip(userName, snetiment):
        graph.add_node(usr, snt=0)
        
    #Conectar nodos de personas retweeteadas
    content_entities = df['entities'].tolist()
    user_id = df['user'].tolist()
    twit = df['text'].tolist()
    for names, author, twit in zip(content_entities, user_id, twit):
        sent = sentimentalAnalis(twit)
        names = eval(str(names))['user_mentions']
        author = eval(str(author))['screen_name']
        for val in names:
            try:
                graph[author][val['screen_name']]["weight"]+=1
                graph[author][val['screen_name']]["sentiment"]+= sent
                
            except:
                graph.add_edge(author,val['screen_name'],weight=1,sentiment=sent)

                #Asignar sentimiento a los enlaces
    for var1,var2,var3 in graph.edges(data=True):
        result = float(var3.get('sentiment')) / float(var3.get('weight'))
        graph[var1][var2]["sentiment"]=result
    return graph



In [10]:
def updateSentimentNode(graph):
    atributeNode = []
    userS = 0
    outDegreeNode = []
    nx.set_node_attributes(graph, atributeNode, 'snt')

    for n in graph.nodes():
        userS = 0
        for outEdges in graph.out_edges(nbunch=n,data=True):
            userS += outEdges[2]['sentiment']
        dout = graph.out_degree(nbunch=n)
        print(n)
        graph.nodes[n]['snt'] = userS/dout if dout>0 else 0
        #atributeNode.append(userS/dout if dout>0 else 0)
    return graph
            

In [11]:
def tweet_clean(tweet):
    #print('Original tweet:', tweet, '\n')
    tweet = str(tweet)
    # Remove HTML special entities (e.g. &amp;)
    tweet_no_special_entities = re.sub(r'\&\w*;', '', tweet)
    tweet_no_special_entities = re.sub(r'\@\w*;', '', tweet)
    tweet_no_special_entities = re.sub(r'\\', '', tweet)
    #print('No special entitites:', tweet_no_special_entities, '\n')
    # Remove tickers
    tweet_no_tickers = re.sub(r'\$\w*', '', tweet_no_special_entities)
    #print('No tickers:', tweet_no_tickers, '\n')
    # Remove hyperlinks
    tweet_no_hyperlinks = re.sub(r'https?:\/\/.*\/\w*', '', tweet_no_tickers)
    #print('No hyperlinks:', tweet_no_hyperlinks, '\n')
    # Remove hashtags
    tweet_no_hashtags = re.sub(r'#\w*', '', tweet_no_hyperlinks)
    #print('No hashtags:', tweet_no_hashtags, '\n')
    # Remove Punctuation and split 's, 't, 've with a space for filter
    tweet_no_punctuation = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet_no_hashtags)
    #print('No punctuation:', tweet_no_punctuation, '\n')
    # Remove https
    tweet_no_https = re.sub(r'https', '', tweet_no_punctuation)
    tweet_no_https = re.sub(r'http', '', tweet_no_punctuation)
    #print('No https:', tweet_no_https, '\n')
    # Remove words with 2 or fewer letters
    tweet_no_small_words = re.sub(r'\b\w{1,2}\b', '', tweet_no_https)
    #print('No small words:', tweet_no_small_words, '\n')
    # Remove whitespace (including new line characters)
    tweet_no_whitespace = re.sub(r'\s\s+', ' ', tweet_no_small_words) 
    tweet_no_whitespace = tweet_no_whitespace.lstrip(' ') # Remove single space remaining at the front of the tweet.
    #print('No whitespace:', tweet_no_whitespace, '\n')
	# Remove characters beyond Basic Multilingual Plane (BMP) of Unicode:
    # tweet_no_emojis = ''.join(c for c in tweet_no_whitespace if c <= '\uFFFF') # Apart from emojis (plane 1), this also removes historic scripts and mathematical alphanumerics (also plane 1), ideographs (plane 2) and more.
    # #print('No emojis:', tweet_no_whitespace, '\n')
    # Tokenize: Change to lowercase, reduce length and remove handles
    tknzr = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True) # reduce_len changes, for example, waaaaaayyyy to waaayyy.
    tw_list = tknzr.tokenize(tweet_no_whitespace)
    #print('Tweet tokenize:', tw_list, '\n')
    # Remove stopwords
    english_stopwords = stopwords.words('english')
    list_no_stopwords = [i for i in tw_list if i not in english_stopwords]
    #print('No stop words:', list_no_stopwords, '\n')
    # 

    # Final filtered tweet
    tweet_filtered =' '.join(list_no_stopwords)

    #print ('Final tweet: ', tweet_filtered)
    
    return(tweet_filtered)

In [12]:
def ldaMethod(data, topics):
    tokenizer = RegexpTokenizer(r'\w+')

    # Create p_stemmer of class PorterStemmer
    p_stemmer = PorterStemmer()

    # list for tokenized twiits in loop
    palabras = []
    for i in data:
        raw = i.lower()
        tokens = tokenizer.tokenize(raw)
        # stem tokens
        stemmed_tokens = [p_stemmer.stem(word) for word in tokens]
        
        # add tokens to list
        palabras.append(stemmed_tokens)
        # twit tokenizado en documento 
    elDictionary = corpora.Dictionary(palabras)

    #token en document-termino matriz
    corpus = [elDictionary.doc2bow(text) for text in palabras]
    
    # Generacion LDA
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=topics, id2word = elDictionary, random_state=5000 ,passes=20)
    modelLDA = ldamodel 
    ldaResult = ldamodel.print_topics(5)
    coherence_model_lda = CoherenceModel(model=ldamodel, texts=palabras, dictionary=elDictionary, coherence='c_v', processes=8)
    coherence_lda = coherence_model_lda.get_coherence()

    return [ldaResult,corpus, elDictionary, modelLDA]

In [13]:
def hlda(corpus, eldictionary, topic, probably_words):
    hdp = HdpModel(corpus, eldictionary)
    topic_info = hdp.print_topics(num_topics=20, num_words=10)
    print (topic_info)

In [14]:
def graficModeLDA(ldaResult,corpus, elDictionary, modelLDA):
    d = elDictionary
    c = corpus
    lda = modelLDA
    data = pyLDAvis.gensim.prepare(lda, c, d)
    data
    
